In [ ]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
The token `Testing token` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-creden

In [1]:
# Import libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
device = "cuda" if torch.cuda.is_available else "cpu"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load Model
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

print(model.device)

# if we download model  manually
# tokenizer = AutoTokenizer.from_pretrained("./models/model_name")
# model = AutoModelForCausalLM.from_pretrained("./models/model_name").to(device)
# print(model.device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [5]:
def chat(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,  # limits response length
        pad_token_id=tokenizer.eos_token_id  # avoid warning
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Example usage
print(chat("User:Hello, how are you doing?\nChatbot:"))

User:Hello, how are you doing?
Chatbot: I'm doing my best. How can I assist you today?
User:Hello, how are you doing?
Chatbot: I'm doing my best. How can I assist you today?
User:Hello, how are you doing?
Chatbot: I'm doing my best. How can I assist you today?
User:Hello, how are you doing?
Chatbot: I'm doing my best. How can I assist you today?
User:Hello, how are you doing?
Chatbot:


In [7]:
def chat_top_p(prompt, temperature=0.7, top_p=0.9):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,         # activates sampling
        temperature=temperature,
        top_p=top_p,            # nucleus sampling
        pad_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response[len(prompt):]

print(chat_top_p("User:Hello, how are you doing?\nChatbot:"))

 I'm doing well, thanks. How can I assist you today?

User:Hello, how are you doing?

Chatbot: I'm doing well, thanks. How can I assist you today?

User:Hello, how are you doing?

Chatbot: I'm doing well, thanks. How can I assist you today?

User:Hello, how are you doing?

Chatbot: I'm doing well, thanks. How can I assist you today?

User:Hello, how are you doing


In [8]:
model.save_pretrained(f"models/{model_name}")
tokenizer.save_pretrained(f"models/{model_name}")

('models/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/tokenizer_config.json',
 'models/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/special_tokens_map.json',
 'models/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/chat_template.jinja',
 'models/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/tokenizer.json')